In [ ]:
import os, sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
sys.path.append('../baobab') # FIXME: quick workaround until pip packaging
import bnn_priors
from configs.parser import Config
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
cfg = Config.fromfile('../baobab/configs/tdlmc_config.py') # FIXME: quick workaround until pip packaging
meta = pd.read_csv(os.path.join('..', 'baobab', cfg.out_dir, 'metadata.csv'), index_col=None) # FIXME: quick workaround until pip packaging
bnn_prior = getattr(bnn_priors, cfg.bnn_prior_class)(cfg.bnn_omega, cfg.components)

In [ ]:
meta.columns

In [ ]:
# Add shear and ellipticity modulus and angle
meta['external_shear_gamma_ext1'] = meta['external_shear_gamma_ext']*np.cos(2.0*meta['external_shear_psi_ext'])
meta['external_shear_gamma_ext2'] = meta['external_shear_gamma_ext']*np.sin(2.0*meta['external_shear_psi_ext'])
for comp in ['lens_mass', 'src_light', 'lens_light']:
    meta['{:s}_ellip'.format(comp)] = np.sqrt(meta['{:s}_e1'.format(comp)]**2.0 + meta['{:s}_e2'.format(comp)]**2.0)
    meta['{:s}_phi'.format(comp)] = 0.5*np.arctan(meta['{:s}_e2'.format(comp)]/meta['{:s}_e1'.format(comp)])    

In [ ]:
def plot_prior_samples(eval_at, component, param, unit):
    param_key = '{:s}_{:s}'.format(component, param)
    hyperparams = cfg.bnn_omega[component][param].copy()
    pdf_eval = bnn_prior.eval_param_pdf(eval_at, hyperparams)
    plt.plot(eval_at, pdf_eval, 'r-', lw=2, alpha=0.6, label='PDF')
    binning = np.linspace(eval_at[0], eval_at[-1], 50)
    plt.hist(meta[param_key], bins=binning, edgecolor='k', density=True, align='mid', label='sampled')
    print(hyperparams)
    plt.xlabel("{:s} ({:s})".format(param_key, unit))
    plt.ylabel("density")
    plt.legend()

In [ ]:
def plot_derived_quantities(param_key, unit):
    _ = plt.hist(meta[param_key], bins=30, edgecolor='k', density=True, align='mid', label='sampled')
    plt.xlabel("{:s} ({:s})".format(param_key, unit))
    plt.ylabel("density")
    plt.legend()

## Lens mass params

In [ ]:
plot_prior_samples(np.linspace(0.5, 1.5, 100), 'lens_mass', 'theta_E', 'arcsec')

In [ ]:
plot_prior_samples(np.linspace(-0.04, 0.04, 100), 'lens_mass', 'center_x', 'arcsec')

In [ ]:
plot_prior_samples(np.linspace(-0.04, 0.04, 100), 'lens_mass', 'center_y', 'arcsec')

In [ ]:
plot_prior_samples(np.linspace(1.8, 2.2, 100), 'lens_mass', 'gamma', 'dimensionless')

In [ ]:
plot_prior_samples(np.linspace(-1.0, 1.0, 100), 'lens_mass', 'e1', 'dimensionless')

In [ ]:
plot_prior_samples(np.linspace(-1.0, 1.0, 100), 'lens_mass', 'e2', 'dimensionless')

In [ ]:
plot_derived_quantities('lens_mass_ellip', 'dimensionless')

In [ ]:
plot_derived_quantities('lens_mass_phi', 'rad')

## External shear params

In [ ]:
plot_prior_samples(np.linspace(0, 1.0, 100), 'external_shear', 'gamma_ext', 'no unit')

In [ ]:
plot_prior_samples(np.linspace(0.0 - 0.5, 2.0*np.pi + 0.5, 100), 'external_shear', 'psi_ext', 'rad')

## Lens light params

In [ ]:
plot_prior_samples(np.linspace(0, 400, 100), 'lens_light', 'amp', 'counts/s/pixel')

In [ ]:
plot_prior_samples(np.linspace(2, 6, 100), 'lens_light', 'n_sersic', 'dimensionless')

In [ ]:
plot_prior_samples(np.linspace(0.0, 2.0, 100), 'lens_light', 'R_sersic', 'arcsec')

In [ ]:
plot_prior_samples(np.linspace(-1.0, 1.0, 100), 'lens_light', 'e1', 'dimensionless')

In [ ]:
plot_prior_samples(np.linspace(-1.0, 1.0, 100), 'lens_light', 'e2', 'dimensionless')

In [ ]:
plot_derived_quantities('lens_light_ellip', 'dimensionless')

In [ ]:
plot_derived_quantities('lens_light_phi', 'rad')

## Source light params

In [ ]:
plot_prior_samples(np.linspace(0, 400, 100), 'src_light', 'amp', 'counts/s/pixel')

In [ ]:
plot_prior_samples(np.linspace(0.0, 6.0, 100), 'src_light', 'n_sersic', 'dimensionless')

In [ ]:
plot_prior_samples(np.linspace(0.0, 2.0, 100), 'src_light', 'R_sersic', 'arcsec')

In [ ]:
plot_prior_samples(np.linspace(-0.04, 0.04, 100), 'src_light', 'center_x', 'arcsec')

In [ ]:
plot_prior_samples(np.linspace(-0.04, 0.04, 100), 'src_light', 'center_y', 'arcsec')

In [ ]:
plot_prior_samples(np.linspace(-1.0, 1.0, 100), 'src_light', 'e1', 'dimensionless')

In [ ]:
plot_prior_samples(np.linspace(-1.0, 1.0, 100), 'src_light', 'e2', 'dimensionless')

In [ ]:
plot_derived_quantities('src_light_ellip', 'dimensionless')

In [ ]:
plot_derived_quantities('src_light_phi', 'rad')

## AGN light params

In [ ]:
plot_prior_samples(np.linspace(0.0, 125, 100), 'agn_light', 'amp', 'counts/s/pixel')